In [3]:
import pyAgrum as gum

In [4]:
# Crear la red bayesiana
bn = gum.BayesNet('SecuritySystem')

In [5]:
# Añadir nodos
http_requests = bn.add(gum.LabelizedVariable('HTTP_Requests', 'HTTP Requests', 2))
attacks = bn.add(gum.LabelizedVariable('Attacks', 'Attacks', 2))
serious_failures = bn.add(gum.LabelizedVariable('Serious_Failures', 'Serious Failures', 2))
firewall = bn.add(gum.LabelizedVariable('Firewall', 'Firewall', 2))
recovery_system = bn.add(gum.LabelizedVariable('Recovery_System', 'Recovery System', 2))

In [6]:
# Añadir arcos
bn.addArc(http_requests, attacks)
bn.addArc(attacks, serious_failures)
bn.addArc(firewall, attacks)
bn.addArc(recovery_system, serious_failures)

In [ ]:
# Definir las tablas de probabilidad condicional (CPTs)
bn.cpt(http_requests).fillWith([0.9, 0.1])  # 90% no ataques, 10% ataques
bn.cpt(attacks)[{'http_requests': 0}] = [0.9, 0.1]  # 90% no ataques si no hay peticiones, 10% ataques
bn.cpt(attacks)[{'http_requests': 1}] = [0.5, 0.5]  # 50% no ataques si hay peticiones, 50% ataques
bn.cpt(serious_failures)[{'attacks': 0}] = [0.99, 0.01]  # 99% no fallos graves si no hay ataques, 1% fallos graves
bn.cpt(serious_failures)[{'attacks': 1}] = [0.5, 0.5]  # 50% no fallos graves si hay ataques, 50% fallos graves
bn.cpt(firewall).fillWith([0.75, 0.25])  # 75% firewall activo, 25% no activo
bn.cpt(recovery_system).fillWith([0.75, 0.25])  # 75% sistema de recuperación activo, 25% no activo

(pyAgrum.Potential@0000024DFB203750) 
  Recovery_System  |
0        |1        |
---------|---------|
 0.7500  | 0.2500  |

In [8]:
# Crear un objeto de inferencia
ie = gum.LazyPropagation(bn)

In [9]:
# Calcular la probabilidad de un fallo grave en cualquier momento
ie.makeInference()
prob_serious_failure = ie.posterior(serious_failures)[1]
print(f"Probabilidad de un fallo grave en cualquier momento: {prob_serious_failure}")

Exception: [pyAgrum] Several evidence/CPT are incompatible together (proba=0): some evidence entered into the Bayes net are incompatible (their joint proba = 0)

In [ ]:
# Calcular la probabilidad de un fallo grave sin cortafuegos
ie.setEvidence({'Firewall': 1})
ie.makeInference()
prob_serious_failure_no_firewall = ie.posterior(serious_failures)[1]
print(f"Probabilidad de un fallo grave sin cortafuegos: {prob_serious_failure_no_firewall}")

In [ ]:
# Función para obtener los nombres de las variables de una red bayesiana
def get_variable_names(bn):
    return [bn.variable(i).name() for i in bn.nodes()]

In [ ]:
# Función para verificar la independencia marginal entre pares de variables
def check_marginal_independence(bn):
    independence_results = {}
    for var1 in bn.nodes():
        for var2 in bn.nodes():
            if var1 != var2:
                independence_results[(bn.variable(var1).name(), bn.variable(var2).name())] = bn.isIndependent(var1, var2)
    return independence_results

In [ ]:
# Obtener nombres de las variables
variable_names = get_variable_names(bn)
print(f"Nombres de las variables: {variable_names}")

In [ ]:
# Verificar independencia marginal entre pares de variables
independence_results = check_marginal_independence(bn)
for vars_pair, is_independent in independence_results.items():
    print(f"{vars_pair}: {'Independientes' if is_independent else 'Dependientes'}")